# Optymalizacja hiperparametrów w bibliotece scipy

Zbiór danych do analizy: https://www.kaggle.com/datasets/prishasawhney/mushroom-dataset

Mamy dane dotyczące grzybów. Model ma za zadanie ocenić, czy grzyb jest jadalny.
Cel biznesowy: Stworzenie aplikacji, która pomoże użytkownikowi w ocenie czy grzyb jest jadalny, poprawiając bezpieczeństwo.

Zmienne:
- Cap Diameter
- Cap Shape
- Gill Attachment
- Gill Color
- Stem Height
- Stem Width
- Stem Color
- Season
- Target Class - Is it edible or not?

In [1]:
import pandas as pd
from scipy.optimize import differential_evolution , minimize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report


In [2]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [3]:
# Pobranie danych
df = pd.read_csv('data/mushroom.csv')

In [ ]:
# nagłówek 
df.head()

In [ ]:
df.shape

In [5]:
# Podział na zbiór treningowy i testowy
train_x, test_x,train_y, test_y = train_test_split(df.drop('class',axis=1),df['class'], test_size=0.2, random_state=1000)

In [6]:
# Podział na zbiór treningowy i testowy
train_x, valid_x,train_y, valid_y = train_test_split(train_x,train_y, test_size=0.2, random_state=1000)

In [7]:
# Pobranie funkcji
from scipy_funkcje import optimization_function

In [27]:
# Granice przeszukiwan
bounds = [(0.00001,1),
          (5,50),
          (0,5)]

## Minimize

In [20]:
# Parametry inicjalne
x0 = [0.3,5,0]

In [22]:
# Minimalizacja
opt_minimize = minimize(fun = optimization_function,
                        x0 = x0,
                        bounds= bounds,
                        args=(train_x,train_y, test_x,test_y),
                        options={'maxiter': 20})

In [ ]:
# Najlepsze parametry
best_params = opt_minimize.x
best_params

In [24]:
# Model 
model_min = HistGradientBoostingClassifier(learning_rate = best_params[0],
                                           max_depth = int(round(best_params[1])),
                                           l2_regularization=int(round(best_params[2]))).fit(train_x,train_y)

In [25]:
# predykcje
valid_pred_m1 = model_min.predict(valid_x)

## Optymalizacja ewolucyjna

In [28]:
# Optymalizacja ewolucyjna
optimization = differential_evolution(func = optimization_function,
                                      bounds=bounds,
                                      args = (train_x,train_y,test_x,test_y),
                                      strategy='best1bin',
                                      maxiter = 10)

In [ ]:
# Najlepsze parametry
best_params_evolution = optimization.x
best_params_evolution

In [30]:
# Model 
model = HistGradientBoostingClassifier(learning_rate = best_params_evolution[0],
                                           max_depth = int(round(best_params_evolution[1])),
                                           l2_regularization=int(round(best_params_evolution[2]))).fit(train_x,train_y)


In [31]:
# predykcje
valid_pred_m2 = model.predict(valid_x)

In [ ]:
# raport klasyfikacji model 1
print(classification_report(valid_y,valid_pred_m1))

In [ ]:
# raport klasyfikacji model 2
print(classification_report(valid_y,valid_pred_m2))